<a href="https://colab.research.google.com/github/as9786/ML-DLPratice/blob/main/%EA%B3%BC%EC%A0%81%ED%95%A9%ED%95%B4%EA%B2%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module

In [1]:
import tensorflow as tf
import numpy as np

# Dataset

In [2]:
cifar10 = tf.keras.datasets.cifar10

In [3]:
(X_train,y_train),(X_test,y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [4]:
X_train, X_test = X_train/255., X_test/255.

In [5]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(1000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).shuffle(1000).batch(32).prefetch(1024)

# Model

In [7]:
EPOCHS = 100

In [8]:
class ConvLNReluBlock(tf.keras.Model):

  def __init__(self,filters,kernel_size):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters,kernel_size,padding='same',use_bias=False)
    self.ln = tf.keras.layers.LayerNormalization()

  def call(self,x,training=False,mask=None):
    x = self.conv(x)
    x = self.ln(x)
    return tf.nn.relu(x)

배치 정규화는 training False, True를 입력해주어야 하지만 층 정규화는 필요 없음

In [9]:
class VGG16(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.conv1_1 = ConvLNReluBlock(16,(3,3))
    self.conv1_2 = ConvLNReluBlock(16,(3,3))
    self.pool1 = tf.keras.layers.MaxPool2D((2,2))

    self.conv2_1 = ConvLNReluBlock(32,(3,3))
    self.conv2_2 = ConvLNReluBlock(32,(3,3))
    self.pool2 = tf.keras.layers.MaxPool2D((2,2))

    self.conv3_1 = ConvLNReluBlock(64,(3,3))
    self.conv3_2 = ConvLNReluBlock(64,(3,3))


    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))
    self.dense2 = tf.keras.layers.Dense(10,activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(0.01))

  def call(self,x,training=False,mask=None):
    x = self.conv1_1(x)
    x = self.conv1_2(x)
    x = self.pool1(x)

    x = self.conv2_1(x)
    x = self.conv2_2(x)
    x = self.pool2(x)

    x = self.flatten(x)
    x = self.dense1(x)
    out = self.dense2(x)

    return out


# 학습

In [10]:
model = VGG16()

In [11]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy')

In [12]:
model.fit(train_ds,validation_data=test_ds,epochs=EPOCHS)

Epoch 1/100
1563/1563 [==============================] - 32s 11ms/step - loss: 2.1989 - val_loss: 1.3400
Epoch 2/100
1563/1563 [==============================] - 17s 11ms/step - loss: 1.2761 - val_loss: 1.2030
Epoch 3/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.1572 - val_loss: 1.0970
Epoch 4/100
1563/1563 [==============================] - 17s 11ms/step - loss: 1.0945 - val_loss: 1.0690
Epoch 5/100
1085/1563 [===================>..........] - ETA: 4s - loss: 1.0542

KeyboardInterrupt: ignored